# Домашнее задание 1. NLTK

## Работу выполнил Ковалев Евгений, студент 4 курса бакалавриата факультета математики НИУ ВШЭ

В данном домашнем задании необходимо было попрактиковаться в работе с NLTK. Из данного текста нужно было выделить наиболее часто встречающиеся словоформы и леммы со стоп-словами и без, найти более-менее осмысленные синонимы и слова с общим контекстом, а также придумать пример на задачу лемматизации, где pymorphy показывает себя лучше mystem.

In [90]:
import re
from nltk import FreqDist, Text, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem

stop_words = stopwords.words('russian')

# 1

*Выберите произвольный достаточно длинный текст.*

Для выполнения задания был выбран текст романа Джерома Сэлинджера «Над пропастью во ржи», доступный по данной ссылке: https://www.e-reading.club/bookreader.php/55486/Selindzher_-_Nad_propast%27yu_vo_rzhi.html

Посмотрим, насколько он велик.

In [91]:
text = ''
with open('the_catcher_in_the_rye_rus.txt') as file:
    for line in file:
        text += line.strip() + ' '
text = text.replace('— ', '')

In [92]:
def tokenization(text):
    tokens = WhitespaceTokenizer().tokenize(text.lower())
    types = FreqDist(tokens)
    return tokens, types

In [93]:
text_tokens, text_types = tokenization(text)
print('number of tokens:', len(text_tokens))
print('number of types:', len(text_types))

number of tokens: 56868
number of types: 13890


Итак, выбранный текст содержит почти 57 тысяч токенов и почти 14 тысяч типов, так что его можно считать достаточно большим и подходящим для анализа.

# 2

*Найдите 20 самых частотных словоформ и лемм со стоп-словами и без стоп-слов.*

Словоформы со стоп-словами:

In [94]:
prog = re.compile('[А-Яа-я\-]+')
l1 = prog.findall(text.lower())
d1 = FreqDist(l1)
print(d1)
print(d1.most_common(20))

<FreqDist with 9637 samples and 56874 outcomes>
[('я', 2114), ('не', 1758), ('и', 1699), ('в', 1284), ('что', 1073), ('он', 871), ('на', 775), ('а', 695), ('но', 636), ('она', 581), ('с', 558), ('у', 514), ('все', 503), ('как', 502), ('меня', 478), ('мне', 476), ('было', 392), ('это', 353), ('ты', 349), ('когда', 322)]


Словоформы без стоп-слов:

In [95]:
l2 = [w for w in l1 if not w in stop_words]
d2 = FreqDist(l2)
print(d2)
print(d2.most_common(20))

<FreqDist with 9487 samples and 29977 outcomes>
[('это', 353), ('говорю', 204), ('очень', 197), ('сказал', 174), ('говорит', 166), ('просто', 128), ('тебе', 124), ('наверно', 114), ('знаю', 112), ('ужасно', 107), ('стал', 95), ('время', 93), ('фиби', 91), ('вообще', 83), ('равно', 77), ('мог', 73), ('хотелось', 73), ('сразу', 70), ('все-таки', 68), ('слово', 68)]


Леммы со стоп-словами:

In [97]:
morph = MorphAnalyzer()
l3 = [morph.parse(token)[0].normal_form for token in l1]
d3 = FreqDist(l3)
print(d3)
print(d3.most_common(20))

<FreqDist with 5270 samples and 56874 outcomes>
[('я', 3126), ('не', 1758), ('и', 1699), ('он', 1452), ('в', 1341), ('что', 1127), ('она', 1038), ('быть', 980), ('на', 775), ('а', 695), ('весь', 682), ('с', 640), ('но', 636), ('ты', 625), ('у', 514), ('говорить', 507), ('как', 502), ('это', 494), ('они', 430), ('сказать', 334)]


Леммы без стоп-слов:

In [98]:
l4 = [w for w in l3 if not w in stop_words]
d4 = FreqDist(l4)
print(d4)
print(d4.most_common(20))

<FreqDist with 5165 samples and 30139 outcomes>
[('весь', 682), ('говорить', 507), ('это', 494), ('сказать', 334), ('мочь', 271), ('знать', 243), ('очень', 197), ('стать', 188), ('ещё', 176), ('свой', 160), ('ничто', 159), ('хотеть', 137), ('который', 136), ('самый', 134), ('просто', 128), ('слово', 122), ('идти', 122), ('понимать', 121), ('наверно', 114), ('спрашивать', 112)]


Видно, что стоп-слова сильно засоряют список наиболее часто встречающихся словоформ и лемм, и при их отбрасывании картина становится менее бессмысленной. Например, в случае словоформ в списке топ-20 частотных появляется словоформа "фиби", которая на самом деле является именем главной героини романа.

# 3

*Найдите пример, в котором pymorphy «выиграет» у mystem.*

При решении данного пункта возникло предположение, что mystem работает намного лучше, чем pymorphy, ибо пример упорно не находился: mystem все время опережал pymorphy в качестве. Однако мои попытки все же увенчались успехом, и я обнаружил, что mystem не идеально справляется с выбором правильного вида (совершенного или несовершенного) глагола. Мой пример — фраза из трека Oxxxymiron — «Накануне»:

«Доигрался ты, старый дурак, вот и вся эпитафия» 

In [99]:
mystem = Mystem()

In [100]:
example = 'доигрался ты старый дурак вот и вся эпитафия'
morph_lemmas = [morph.parse(w)[0].normal_form for w in example.split()]
mystem_lemmas = mystem.lemmatize(example)
print('pymorphy:', ' '.join(morph_lemmas))
print('mystem:', ''.join(mystem_lemmas))

pymorphy: доиграться ты старый дурак вот и весь эпитафия
mystem: доигрываться ты старый дурак вот и весь эпитафия



В результате обработки данной цитаты двумя лемматизаторами видно, что pymorphy справился лучше — в данном контексте глагол «доиграться» совершенного вида лучше передает смысл предложения, чем глагол «доигрываться» несовершенного вида.

# 4

*Найдите осмысленные синонимы в тексте, а также слова с общим контекстом.*

In [101]:
nltk_text = Text(word_tokenize(text))
print(nltk_text)

<Text: 1 Если вам на самом деле хочется услышать...>


Более-менее осмысленные синонимы:

«вам» и «тебе»

In [135]:
nltk_text.similar('вам')

тебе мне ей ничего и она уже бы ему он ее сразу ты тебя вы я не говорю
это почти


«ничуть» и «ничего»

In [181]:
nltk_text.similar('ничуть')

ничего и даже уже никогда вовсе совершенно никак чуть почти совсем все
его конечно вам я мне бы этого меня


«голливуд» и «кино»

In [249]:
nltk_text.similar('голливуд')

кино пэнси субботу общем лифчик истории шкаф восторге пижамах носу
столовую году капелле комнату гости зеркало отпуск мейне семье гольф


Слова с общим контекстом:

«он» и «она»

In [273]:
nltk_text.common_contexts(['он', 'она'])

что_уже а_и что_меня вдруг_меня равно_бы что_действительно что_просто
но_все но_всегда тут_вдруг что_не что_даже а_говорит что_тебе но_мне
но_не и_вдруг и_не а_не что_ужасно


«сказал» и «говорю»

In [307]:
nltk_text.common_contexts(['сказал', 'говорю'])

не_но галлахер_я и_что я_нет я_в да_и


«очень» и «слишком»

In [326]:
nltk_text.common_contexts(['очень', 'слишком'])

сейчас_поздно уж_не
